# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [220]:
# Dependencies and Setup
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [221]:
# CitiesKQ CSV File (Path) and read the CSV File
citieskq_path = "/Users/kellenquinn/Desktop/Python-api-challenge/Resources/citieskq.csv"
citieskq_df = pd.read_csv(citieskq_path)


In [222]:
#Display CitiesKQ 
citieskq_df

,Unnamed: 0,city id,city,country,lat,lon,cloudiness %,humidity %,max temp (F),wind speed (mph),date
0,0,2126123,Chokurdakh,RU,70.6333,147.9167,21,86,11.73,1.14,1619235388
1,1,1006984,East London,ZA,-33.0153,27.9116,20,88,59.00,4.61,1619235388
2,2,1733782,Labuan,MY,5.2767,115.2417,40,83,82.40,2.30,1619235388
3,3,3833367,Ushuaia,AR,-54.8000,-68.3000,75,93,42.80,14.97,1619235103
4,4,6620339,Karratha,AU,-20.7377,116.8463,0,18,90.36,16.15,1619235389
...,...,...,...,...,...,...,...,...,...,...,...
553,553,2400547,Gamba,GA,-2.6500,10.0000,100,92,75.70,3.60,1619235594
554,554,2655785,Beverley,GB,53.8459,-0.4233,75,87,43.00,1.14,1619235594
555,555,2082539,Merauke,ID,-8.4667,140.3333,46,75,84.13,5.70,1619235549
556,556,172515,Abu Samrah,SY,35.3029,37.1841,67,28,67.69,4.52,1619235596


In [223]:
#Drop Column Unnamed: 0
citieskq_df = citieskq_df.drop(columns=['Unnamed: 0'])
citieskq_df

,city id,city,country,lat,lon,cloudiness %,humidity %,max temp (F),wind speed (mph),date
0,2126123,Chokurdakh,RU,70.6333,147.9167,21,86,11.73,1.14,1619235388
1,1006984,East London,ZA,-33.0153,27.9116,20,88,59.00,4.61,1619235388
2,1733782,Labuan,MY,5.2767,115.2417,40,83,82.40,2.30,1619235388
3,3833367,Ushuaia,AR,-54.8000,-68.3000,75,93,42.80,14.97,1619235103
4,6620339,Karratha,AU,-20.7377,116.8463,0,18,90.36,16.15,1619235389
...,...,...,...,...,...,...,...,...,...,...
553,2400547,Gamba,GA,-2.6500,10.0000,100,92,75.70,3.60,1619235594
554,2655785,Beverley,GB,53.8459,-0.4233,75,87,43.00,1.14,1619235594
555,2082539,Merauke,ID,-8.4667,140.3333,46,75,84.13,5.70,1619235549
556,172515,Abu Samrah,SY,35.3029,37.1841,67,28,67.69,4.52,1619235596


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [224]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [225]:
# Store 'Lat' and 'Lng' into  locations 
locations = citieskq_df[["lat", "lon"]].astype(float)

In [226]:
# Convert Humdity % to float
humidity_percent = citieskq_df["humidity %"].astype(float)

In [227]:
 # Create a Humidity Percent Layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_percent, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [228]:
#Defining the Perfect City and Filtering Out Destinations
perfectcities_df = citieskq_df.loc[(citieskq_df['cloudiness %'] <50) & (citieskq_df['max temp (F)'] >60) & (citieskq_df['max temp (F)'] <=90) & (citieskq_df['humidity %'] <30)].dropna()

perfectcities_df

,city id,city,country,lat,lon,cloudiness %,humidity %,max temp (F),wind speed (mph),date
10,2070998,Geraldton,AU,-28.7667,114.6000,3,21,86.00,5.75,1619235391
19,2175819,Biloela,AU,-24.4167,150.5000,20,24,82.18,4.25,1619235395
53,1835848,Seoul,KR,37.5683,126.9778,0,26,71.60,8.05,1619235165
55,103630,Najrān,SA,17.4924,44.1277,0,26,68.00,4.61,1619235408
57,1174167,Khanpur,PK,28.6453,70.6567,0,23,85.73,7.00,1619235409
70,2450173,Taoudenni,ML,22.6783,-3.9836,0,8,83.57,8.41,1619235413
107,5546220,Saint George,US,37.1041,-113.5841,1,12,71.60,6.91,1619235428
118,2381334,Atar,MR,20.5169,-13.0499,0,17,75.07,6.51,1619235413
152,5861897,Fairbanks,US,64.8378,-147.7164,1,24,60.80,6.91,1619235313
159,1280037,Shache,CN,38.4167,77.2406,10,10,63.10,2.51,1619235446


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [229]:
#Create Hotel DF and add Hotel Name Column
hotel_df = pd.DataFrame(perfectcities_df, columns=['city', 'country', 'lat', 'lon'])
hotel_df["hotel name"] = "" 
hotel_df

,city,country,lat,lon,hotel name
10,Geraldton,AU,-28.7667,114.6000,
19,Biloela,AU,-24.4167,150.5000,
53,Seoul,KR,37.5683,126.9778,
55,Najrān,SA,17.4924,44.1277,
57,Khanpur,PK,28.6453,70.6567,
70,Taoudenni,ML,22.6783,-3.9836,
107,Saint George,US,37.1041,-113.5841,
118,Atar,MR,20.5169,-13.0499,
152,Fairbanks,US,64.8378,-147.7164,
159,Shache,CN,38.4167,77.2406,


In [232]:
#Loop through the list of cities and perform data request
hotel_name_list=[]
lat = hotel_df['lat']
lon = hotel_df['lon']

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    query_url=f'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # update address key value
    lat = row["lat"]
    lon = row["lon"]
    params["location"] = f"{lat},{lon}"

    #make request
    response=requests.get(query_url, params=params).json()
  
    try:
       hotel_df.loc[index, "hotel name"] = response["results"][0]["name"]
    except:
        print(params["location"])
        print("Hotel not found")           

22.6783,-3.9836
Hotel not found
18.4411,1.4078
Hotel not found
20.5022,-10.0711
Hotel not found
32.8333,66.0
Hotel not found


In [233]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [235]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))